In [1]:
%load_ext autoreload
%autoreload 2
import sys, os, torch, datetime, time
import numpy as np
sys.path.append("..")
import rfi_ml_new_train_each_epoch
import matplotlib.pyplot as plt

In [2]:
Ntrain = 100000 #the size of the training set
Ntest = 10
sigma_low = 20
sigma_high = 50
Nepochs = 30
Nrfi = 1
Np = 2**10 #1k
z_dim = 16
Nfft = Np // 2 + 1
#Pk = np.ones(Nfft)
Pk = None
#lamb = 0.9 #lambda, scaling down Gaussianized amplitude
np.random.seed(0) #for debugging
torch.manual_seed(0) #more debugging
start_time = time.time()



In [3]:
n = rfi_ml_new_train_each_epoch.RFIDetect_each_epoch(Np = Np, Ntrain = Ntrain, Ntest = Ntest, Pk = Pk, Nepochs = Nepochs, z_dim = z_dim)

In [4]:
n.train()

[  0/30][  0/100000]	Loss: 0.0014318469
[  0/30][100/100000]	Loss: 0.0010497014
[  0/30][200/100000]	Loss: 0.0010777297
[  0/30][300/100000]	Loss: 0.0010298422
[  0/30][400/100000]	Loss: 0.0010538502
[  0/30][500/100000]	Loss: 0.0010030040
[  0/30][600/100000]	Loss: 0.0010234979
[  0/30][700/100000]	Loss: 0.0011260480
[  0/30][800/100000]	Loss: 0.0011332739
[  0/30][900/100000]	Loss: 0.0010586701
[  0/30][1000/100000]	Loss: 0.0011788273
[  0/30][1100/100000]	Loss: 0.0010520753
[  0/30][1200/100000]	Loss: 0.0010469102
[  0/30][1300/100000]	Loss: 0.0010575689
[  0/30][1400/100000]	Loss: 0.0010574781
[  0/30][1500/100000]	Loss: 0.0010200407
[  0/30][1600/100000]	Loss: 0.0010595316
[  0/30][1700/100000]	Loss: 0.0011047915
[  0/30][1800/100000]	Loss: 0.0010749309
[  0/30][1900/100000]	Loss: 0.0011155437
[  0/30][2000/100000]	Loss: 0.0010777715
[  0/30][2100/100000]	Loss: 0.0009853439
[  0/30][2200/100000]	Loss: 0.0009262447
[  0/30][2300/100000]	Loss: 0.0010154468
[  0/30][2400/100000]	Loss

[  6/30][1450/100000]	Loss: 0.0004125374
[  6/30][1550/100000]	Loss: 0.0004118659
[  6/30][1650/100000]	Loss: 0.0004195105
[  6/30][1750/100000]	Loss: 0.0004182155
[  6/30][1850/100000]	Loss: 0.0004187807
[  6/30][1950/100000]	Loss: 0.0004218018
[  6/30][2050/100000]	Loss: 0.0004059913
[  6/30][2150/100000]	Loss: 0.0004045510
[  6/30][2250/100000]	Loss: 0.0004269466
[  6/30][2350/100000]	Loss: 0.0004196438
[  6/30][2450/100000]	Loss: 0.0004255556
[  6/30][2550/100000]	Loss: 0.0004104208
[  6/30][2650/100000]	Loss: 0.0004161714
[  6/30][2750/100000]	Loss: 0.0004148522
[  6/30][2850/100000]	Loss: 0.0004246188
[  6/30][2950/100000]	Loss: 0.0004135554
[  6/30][3050/100000]	Loss: 0.0004050369
[  7/30][ 25/100000]	Loss: 0.0004101745
[  7/30][125/100000]	Loss: 0.0004205864
[  7/30][225/100000]	Loss: 0.0004002383
[  7/30][325/100000]	Loss: 0.0004139033
[  7/30][425/100000]	Loss: 0.0004103720
[  7/30][525/100000]	Loss: 0.0004120808
[  7/30][625/100000]	Loss: 0.0004185446
[  7/30][725/100000]	Lo

[ 12/30][2900/100000]	Loss: 0.0004101320
[ 12/30][3000/100000]	Loss: 0.0004087403
[ 12/30][3100/100000]	Loss: 0.0003933517
[ 13/30][ 75/100000]	Loss: 0.0003938451
[ 13/30][175/100000]	Loss: 0.0003935153
[ 13/30][275/100000]	Loss: 0.0003917913
[ 13/30][375/100000]	Loss: 0.0003928823
[ 13/30][475/100000]	Loss: 0.0004037163
[ 13/30][575/100000]	Loss: 0.0003901388
[ 13/30][675/100000]	Loss: 0.0003942982
[ 13/30][775/100000]	Loss: 0.0004097940
[ 13/30][875/100000]	Loss: 0.0003941569
[ 13/30][975/100000]	Loss: 0.0003882553
[ 13/30][1075/100000]	Loss: 0.0003906170
[ 13/30][1175/100000]	Loss: 0.0004023142
[ 13/30][1275/100000]	Loss: 0.0003949339
[ 13/30][1375/100000]	Loss: 0.0003921127
[ 13/30][1475/100000]	Loss: 0.0003885282
[ 13/30][1575/100000]	Loss: 0.0003915763
[ 13/30][1675/100000]	Loss: 0.0003940102
[ 13/30][1775/100000]	Loss: 0.0004070825
[ 13/30][1875/100000]	Loss: 0.0003945151
[ 13/30][1975/100000]	Loss: 0.0003897177
[ 13/30][2075/100000]	Loss: 0.0003984200
[ 13/30][2175/100000]	Loss

[ 19/30][1225/100000]	Loss: 0.0003878516
[ 19/30][1325/100000]	Loss: 0.0003868214
[ 19/30][1425/100000]	Loss: 0.0003878328
[ 19/30][1525/100000]	Loss: 0.0003911340
[ 19/30][1625/100000]	Loss: 0.0003889619
[ 19/30][1725/100000]	Loss: 0.0003866975
[ 19/30][1825/100000]	Loss: 0.0003843615
[ 19/30][1925/100000]	Loss: 0.0003877448
[ 19/30][2025/100000]	Loss: 0.0003914361
[ 19/30][2125/100000]	Loss: 0.0003892801
[ 19/30][2225/100000]	Loss: 0.0003931701
[ 19/30][2325/100000]	Loss: 0.0003780317
[ 19/30][2425/100000]	Loss: 0.0003977040
[ 19/30][2525/100000]	Loss: 0.0003881359
[ 19/30][2625/100000]	Loss: 0.0003880512
[ 19/30][2725/100000]	Loss: 0.0003858374
[ 19/30][2825/100000]	Loss: 0.0003877771
[ 19/30][2925/100000]	Loss: 0.0003767700
[ 19/30][3025/100000]	Loss: 0.0003928900
[ 20/30][  0/100000]	Loss: 0.0003953741
[ 20/30][100/100000]	Loss: 0.0003836469
[ 20/30][200/100000]	Loss: 0.0003942099
[ 20/30][300/100000]	Loss: 0.0003848006
[ 20/30][400/100000]	Loss: 0.0003900784
[ 20/30][500/100000]	

[ 25/30][2675/100000]	Loss: 0.0003778788
[ 25/30][2775/100000]	Loss: 0.0003997039
[ 25/30][2875/100000]	Loss: 0.0003839502
[ 25/30][2975/100000]	Loss: 0.0003856895
[ 25/30][3075/100000]	Loss: 0.0003873181
[ 26/30][ 50/100000]	Loss: 0.0003832749
[ 26/30][150/100000]	Loss: 0.0003836461
[ 26/30][250/100000]	Loss: 0.0003901753
[ 26/30][350/100000]	Loss: 0.0003976030
[ 26/30][450/100000]	Loss: 0.0003850756
[ 26/30][550/100000]	Loss: 0.0003870258
[ 26/30][650/100000]	Loss: 0.0003853883
[ 26/30][750/100000]	Loss: 0.0003796843
[ 26/30][850/100000]	Loss: 0.0003998547
[ 26/30][950/100000]	Loss: 0.0003831918
[ 26/30][1050/100000]	Loss: 0.0003778924
[ 26/30][1150/100000]	Loss: 0.0003897670
[ 26/30][1250/100000]	Loss: 0.0003820792
[ 26/30][1350/100000]	Loss: 0.0003825924
[ 26/30][1450/100000]	Loss: 0.0003836825
[ 26/30][1550/100000]	Loss: 0.0003782816
[ 26/30][1650/100000]	Loss: 0.0003799102
[ 26/30][1750/100000]	Loss: 0.0003758645
[ 26/30][1850/100000]	Loss: 0.0003927608
[ 26/30][1950/100000]	Loss

In [5]:
g_test_array = torch.stack([torch.from_numpy(n.getGaussian()) for i in range(Ntest)])
rfi_array = np.zeros((Ntest, Np), dtype=float)
for i in range(Ntest):
    rfi_timestream = np.zeros(Np)
    for j in range(Nrfi):
        rfi_timestream += n.getNonGaussianLocalized(sigma=(sigma_low, sigma_high)) 
    rfi_array[i,:] += rfi_timestream
ng_test_array = torch.from_numpy(rfi_array)

testout = n.evaluate(g_test_array, ng_test_array)

Epochs:  30
N tests:  10
Length of timestream:  1024
Timestream Obs-Expect RMS:  [0.00473005 0.00415681 0.0031807  0.00477219 0.00582721 0.00350244
 0.00499337 0.00304614 0.00954767 0.0040169 ]
Avg RMS for all tests:  0.004777346367895993 

Fraction of RFI power remaining:  [0.06056262 0.01788073 0.03231263 0.0377376  0.03374703 0.0357819
 0.03573715 0.00965215 0.19506671 0.08469238]
Remainder as fraction of signal power:  [0.0684395  0.05099656 0.02559791 0.07008802 0.100136   0.03578606
 0.06792022 0.02508598 0.24002599 0.04095279]
Avg of RFI power remaining:  0.05431708857848098
Avg amp of Gaussian signal:  0.015135366948829877


In [6]:
n.plot_eval(testout, g_test_array, ng_test_array)

Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_0.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_0.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_1.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_1.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_2.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_2.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_3.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_3.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_4.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_4.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_5.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_5.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_6.png
Saving file...rfi_ml/2021-12-13_16-31-09_overplot_test_6.png
Saving file...rfi_ml/2021-12-13_16-31-09_epoch_0000030_test_7.png
Saving file...rfi_ml/2021-12-13_16-31-09_over

In [7]:
print("Runtime: ", time.time()-start_time)

Runtime:  796.4114880561829
